In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import tensorflow_datasets as tfds
import tensorflow as tf
from keras import layers, models, optimizers, Input, Model
from keras.models import load_model
import tensorflow_addons as tfa

from malleable_network import GeneticNetwork, MalleableLayer, TerminalLayer
from genetic_training import EvolutionStructure, create_starter_population_entry

2025-01-19 23:21:34.689762: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWa

In [2]:
def plot_metrics_after_training(history, added_title_text=""):
    # ['loss', 'acc', 'f1_score', 'precision', 'recall', 'auc', \
    # 'val_loss', 'val_acc', 'val_f1_score', 'val_precision', 'val_recall', 'val_auc']

    for metric_name in ['loss', 'acc', 'f1_score', 'precision', 'recall', 'auc']:
        training_metric = history.history[metric_name]
        val_metric = history.history[f"val_{metric_name}"]
        epochs = range(1, len(training_metric)+1)

        plt.plot(epochs, training_metric, 'bo', label=f'Training {metric_name}')
        plt.plot(epochs, val_metric, 'b', label=f'Validation {metric_name}')
        plt.title(f'Training and validation {metric_name}{added_title_text}')
        plt.legend()
        plt.show()


def preprocess(sample, image_size, num_classes):
    image = tf.image.resize(sample['image'], image_size) / 255.0  # Normalize
    category = tf.one_hot(sample['label'], depth=num_classes)
    return image, category

def plot_prediction_result(test_image):
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(test_image)
    plt.title("Image")
    plt.axis("off")

    plt.show()

In [3]:
BATCH_SIZE=32

In [4]:
if False:

    dataset, info = tfds.load('cifar10', split=['train', 'test'], with_info=True, as_supervised=False)
    OUTPUT_SIZE = 10
    IMG_SIZE = (32, 32)
    PATCH_NUM = (4,4)
    DATASET_NAME = 'CIFAR-10'

    # dataset, info = tfds.load('cifar100', split=['train', 'test'], with_info=True, as_supervised=False)
    # OUTPUT_SIZE = 100
    # IMG_SIZE = (32, 32)
    # PATCH_NUM = (2,2)
    # DATASET_NAME = 'CIFAR-100'

    # tfds checksum does not pass for this one for some reason
    # dataset, info = tfds.load('caltech101', split=['train', 'test'], with_info=True, as_supervised=False)
    # # OUTPUT_SIZE = 101
    # # IMG_SIZE = (128, 128)
    # PATCH_NUM = (8,8)
    # # DATASET_NAME = 'caltech 101'

    # dataset, info = tfds.load('imagenette', split=['train', 'validation'], with_info=True, as_supervised=False)
    # OUTPUT_SIZE = 10
    # IMG_SIZE = (160, 160)
    # PATCH_NUM = (10,10)
    # DATASET_NAME = 'tiny imagenet'

    # all_imgnet, info = tfds.load('imagenet2012', split='train', shuffle_files=True, as_supervised=False)
    # dataset = all_imgnet.take(5_000)
    # # IMG_SIZE = (128, 128)
    # PATCH_NUM = (8,8)
    # OUTPUT_SIZE=None
    # DATASET_NAME = 'imagenet'


    train_dataset = dataset[0]
    test_dataset = dataset[1]

    for sample in train_dataset.take(4):
        # print(list(sample.keys()))
        # print(sample['id'])
        # print(sample['image'][:5])
        # print(sample['label'])
        image = tf.image.convert_image_dtype(sample['image'], tf.float32)
        print(f"Image shape: {image.shape}")
        print(f"Label (class): {sample['label']}")

    full_train_data = train_dataset.map(lambda sample: preprocess(sample, IMG_SIZE, OUTPUT_SIZE)).shuffle(1024).batch(BATCH_SIZE)

    validation_split = 0.13
    num_train = int((1 - validation_split) * info.splits['train'].num_examples / BATCH_SIZE)
    print(f"number training batches:   {num_train}")
    print(f"number validation batches: {int(info.splits['train'].num_examples / BATCH_SIZE - num_train)}")

    train_data = full_train_data.take(num_train).prefetch(tf.data.AUTOTUNE)
    val_data = full_train_data.skip(num_train).prefetch(tf.data.AUTOTUNE)

    test_data = test_dataset.map(lambda sample: preprocess(sample, IMG_SIZE, OUTPUT_SIZE)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [5]:
# Load the data
data = pd.read_csv('titanic-model/train.csv')

# Fill missing values (this is just an example; you may use other imputation methods)
data['Age'].fillna(data['Age'].median(), inplace=True)
# data['Fare'].fillna(data['Fare'].median(), inplace=True)
# data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# One-hot encode categorical variables
categorical_features = ['Sex', 'Embarked']
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(data[categorical_features])

# Normalize numerical features
numerical_features = ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']
scaler = StandardScaler()
normalized_features = scaler.fit_transform(data[numerical_features])

# Combine features
X = np.concatenate([encoded_features, normalized_features], axis=1)
y = data['Survived'].values

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

INPUT_SHAPE = X_train.shape[1:]
OUTPUT_SIZE = 1

/Users/afalcignombp/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
population = [
    create_starter_population_entry(GeneticNetwork(
        input_shape=INPUT_SHAPE,
        output_features=OUTPUT_SIZE,
        output_activation_str="sigmoid",
    ))
    for _ in range(10)
]

evolution = EvolutionStructure(
    population,
    X_train,
    y_train,
    X_val,
    y_val,
    optimizer_str='adam',
    loss_str='binary_crossentropy'
)

In [7]:
EPOCHS=20

for _ in range(18):
    evolution.iterate_population(train_epochs=EPOCHS)

print("\n   <All done with iterating>")




****************************************************************************************************

<0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0><0>
network rank None (done with 0 epochs)
network fitness None

Genetic Network structure:
MalleableLayer(sequential=True)
Output Layer: Dense(units=1, activation=sigmoid)


<1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1><1>
network rank None (done with 0 epochs)
network fitness None

Genetic Network structure:
MalleableLayer(sequential=True)
Output Layer: Dense(units=1, activation=sigmoid)


<2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2><2>
network rank None (done with 0 epochs)
network fitness None

Genetic Network structure:
MalleableLayer(sequential=True)
Output Layer: Dense(units=1, activation=sigmoid)


<3><3><3><3><3><3><3><3><3><3><3><3><3><3><3><3><3><3><3><3>
network rank None (done with 0 epochs)
network fitness None

Genetic Network structure:
MalleableLayer(sequential=True)
Output Layer: Dense

KeyError: 'stats'

In [ ]:
evolution.train_population(epochs=EPOCHS)
evolution.calculate_fitness()
evolution.rank_population()
for d in evolution.population:
    network, epochs_done, rank, fitness, stats = d['network'], d['training_reps'], d['rank'], d['fitness'], d['stats']
    print()
    print("-*"*80 + "-")
    print(f"\nNetwork Ranked #{rank+1}:")
    print(f"With a fitness of {fitness} after {epochs_done} epochs...")
    network.print_structure()


print(evolution.all_stats)
print("\n best fitness stats over iterations:")
print(evolution.best_stats)


6/6 [==============================] - 0s 2ms/step - loss: 0.4105 - acc: 0.8101 - precision_99: 0.7941 - recall_99: 0.7297 - auc_99: 0.8875
{'network': <malleable_network.GeneticNetwork object at 0x7f77691dd340>, 'training_reps': 50, 'rank': None, 'fitness': -0.410337895154953}
{'network': <malleable_network.GeneticNetwork object at 0x7f77ca6399a0>, 'training_reps': 50, 'rank': 0, 'fitness': -0.41036805510520935}
{'network': <malleable_network.GeneticNetwork object at 0x7f77a19feb20>, 'training_reps': 50, 'rank': None, 'fitness': -0.41054126620292664}
{'network': <malleable_network.GeneticNetwork object at 0x7f77ca28aca0>, 'training_reps': 50, 'rank': 1, 'fitness': -0.4107319116592407}
{'network': <malleable_network.GeneticNetwork object at 0x7f77d99e6e20>, 'training_reps': 50, 'rank': 2, 'fitness': -0.41119739413261414}
{'network': <malleable_network.GeneticNetwork object at 0x7f776915dcd0>, 'training_reps': 50, 'rank': 3, 'fitness': -0.4114128053188324}
{'network': <malleable_network

In [ ]:
model = GeneticNetwork(input_shape=INPUT_SHAPE, output_features=OUTPUT_SIZE, output_activation_str='sigmoid', build=False)
model.malleable_layer = MalleableLayer(
    left=TerminalLayer(force_dimension=1),
    right=MalleableLayer(
        left=TerminalLayer(force_dimension=1),
        right=None,
        sequential=True
    ),
    sequential=True
)
model.build((None,) + model.orig_input_shape)
print(model.summary())
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'acc',
        # tfa.metrics.F1Score(num_classes=OUTPUT_SIZE, average='weighted'),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.AUC()
    ]
)

model.fit(
    X_train,
    y_train,
    epochs=1
)

dummy_input = tf.random.normal((1,) + model.orig_input_shape)  # Example dummy input
print(model(dummy_input))  # Perform a dummy pass to initialize everything

Model: "genetic_network_100"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_221 (Dense)           multiple                  64        
                                                                 
 malleable_layer_458 (Mallea  multiple                 1188      
 bleLayer)                                                       
                                                                 
Total params: 1,252
Trainable params: 1,252
Non-trainable params: 0
_________________________________________________________________
None
23/23 [==============================] - 1s 2ms/step - loss: 0.7237 - acc: 0.4424 - precision_100: 0.3920 - recall_100: 0.8731 - auc_100: 0.5612
tf.Tensor([[0.43507433]], shape=(1, 1), dtype=float32)
